In [ ]:
from langchain_core.documents import Document


documents = [
    Document(
        page_content="犬は忠誠心と友好性で知られる素晴らしい仲間です。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="猫は独立したペットで、しばしば自分のスペースを楽しむことが好きです。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="金魚は初心者に人気のあるペットで、比較的簡単なケアが必要です。",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="オウムは人間の言葉を模倣することができる知的な鳥です。",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="ウサギは社交的な動物で、たくさんのスペースを必要とします。",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
vectorstore.similarity_search("猫")

In [ ]:
await vectorstore.asimilarity_search("猫")

In [ ]:
vectorstore.similarity_search_with_score("猫")

In [ ]:
embedding = OpenAIEmbeddings().embed_query("猫")

vectorstore.similarity_search_by_vector(embedding)

In [ ]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=2)
retriever.batch(["cat", "shark"])

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

In [ ]:
retriever

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


llm = ChatOpenAI(model="gpt-4o-mini")

message = """与えられたコンテキストだけを使って質問に答えてください。

{question}

コンテキスト:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("猫について教えてください")
response.content